# Pathology 전처리 하기

In [1]:
## Path관련 library
from pathlib import Path  # data path관련

## WSI관련 library
import openslide  # WSI 읽어오는 library
import cv2  # 기본적인 CV library
import seaborn as sns  # data analysis library

## etc
import pandas as pd  # csv, dataframe 사용 library
import numpy as np
import matplotlib.pyplot as plt  # image visulization library

## color normalize
import staintools  # spams 설치가 필요한데 python 3.9 버전 이하에서만 설치가능함

In [2]:
# 우선적으로 datset을 읽어와줍니다.
class cfg:
    # Location of the training images
    BASE_PATH = "/mnt/d/Data/medical/panda"
    # image and mask directories
    data_dir = f"{BASE_PATH}/train_images"
    mask_dir = f"{BASE_PATH}/train_label_masks"

In [3]:
# csv 파일을 읽어 data의 경로를 확인합니다
train = pd.read_csv(f"{cfg.BASE_PATH}/train.csv")
train.head()

,image_id,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0


In [ ]:
train["img_path"] = train["image_id"].apply(lambda x: Path(cfg.data_dir) / (x + ".tiff"))
train.head()

,image_id,data_provider,isup_grade,gleason_score,img_path
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0,/mnt/d/Data/medical/panda/train_images/0005f7a...
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0,/mnt/d/Data/medical/panda/train_images/000920a...
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4,/mnt/d/Data/medical/panda/train_images/0018ae5...
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4,/mnt/d/Data/medical/panda/train_images/001c62a...
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0,/mnt/d/Data/medical/panda/train_images/001d865...


In [ ]:
train["img_path"][0]

PosixPath('/mnt/d/Data/medical/panda/train_images/0005f7aaab2800f6170c399693a96917.png')

얻어지는 level에 따라서 size가 달라지며 pixel간에 거리가 다르다

In [7]:
train.loc[0, "img_path"]

PosixPath('/mnt/d/Data/medical/panda/train_images/0005f7aaab2800f6170c399693a96917.png')

In [ ]:
image = openslide.OpenSlide(train.loc[0, "img_path"])  # openslid로 tiff 읽기
max_level = image.level_count  # 가장 해상도가 적은 image 순서 가져오기
print(image.level_dimensions, image.level_count)
level = max_level - 3
small_patch = image.read_region((7100, 10000), level, (1024, 1024))  # r

level = max_level - 2
minum_patch = image.read_region((7100 // pow(2, 1), 10000 // pow(2, 1)), level, (1024, 1024))  # r

level = max_level - 1
min_size = image.level_dimensions[level]
large_patch = image.read_region((7100 // pow(2, 3), 10000 // pow(2, 3)), level, (1024, 1024))  # r

AttributeError: 'PngImageFile' object has no attribute 'level_count'

openslide read region에 방법  
location : 영상이 시작되어지는 위치  
level : Pyramid 영상중에 어떠한 Level를 선택할지

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(15, 15))
ax[0].imshow(small_patch)
ax[1].imshow(minum_patch)
ax[2].imshow(large_patch)

# 전처리

### tile(patching)

In [ ]:
import skimage # 빠른 이미지 load를 위해서 사용
import openslide 
%time image_a = openslide.OpenSlide(train.loc[0,'img_path'])
%time image_b = skimage.io.MultiImage(str(train.loc[0,'img_path']))


Patch로 자르기 이전에 pathology image에서는 white즉 흰공간이 너무 많다.
이를 제거해주는 간편한 코드를 사용해보자.

In [ ]:
level = max_level - 1
min_size = image_a.level_dimensions[level]
large_patch = image_a.read_region((0, 0), level, min_size)  # r

In [ ]:
## int8로 되어있음.
fig, ax = plt.subplots(1, 3, figsize=(10, 2))
sns.histplot(np.array(large_patch)[..., 0].ravel(), ax=ax[0])
sns.histplot(np.array(large_patch)[..., 1].ravel(), ax=ax[1])
sns.histplot(np.array(large_patch)[..., 2].ravel(), ax=ax[2])

In [ ]:
## crop white region
def crop_white(image: np.ndarray) -> np.ndarray:
    assert image.shape[2] == 3
    assert image.dtype == np.uint8
    (ys,) = (image.min((1, 2)) != 255).nonzero()
    (xs,) = (image.min(0).min(1) != 255).nonzero()
    if len(xs) == 0 or len(ys) == 0:
        return image
    return image[ys.min() : ys.max() + 1, xs.min() : xs.max() + 1]

In [ ]:
rm_white_img = crop_white(np.array(large_patch)[..., :3])
fig, ax = plt.subplots(1, 2, figsize=(10, 2))
ax[0].imshow(large_patch)
ax[1].imshow(rm_white_img)

In [ ]:
import math
import cv2

sz = 256
pad = 128
N = 9


def tile(img):
    shape = img.shape
    pad0, pad1 = (sz - shape[0] % sz) % sz, (sz - shape[1] % sz) % sz
    img = np.pad(img, [[pad0 // 2, pad0 - pad0 // 2], [pad1 // 2, pad1 - pad1 // 2], [0, 0]], constant_values=255)
    print([[pad0 // 2, pad0 - pad0 // 2], [pad1 // 2, pad1 - pad1 // 2], [0, 0]])
    img = img.reshape(img.shape[0] // sz, sz, img.shape[1] // sz, sz, 3)
    img = img.transpose(0, 2, 1, 3, 4).reshape(-1, sz, sz, 3)
    if len(img) < N:
        img = np.pad(img, [[0, N - len(img)], [0, 0], [0, 0], [0, 0]], constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0], -1).sum(-1))[:N]
    img = img[idxs]
    return img


def concate_images(img):
    assert len(img.shape) == 4
    assert img.shape[0] == N
    ax_size = int(math.sqrt(N))
    order_index = np.arange(0, N).reshape(ax_size, ax_size)
    hconcat = [cv2.hconcat(img[i]) for i in order_index]
    con_img = cv2.vconcat(hconcat)
    return con_img

In [ ]:
remove_image_b = crop_white(np.array(image_b)[0])  # white 의 값을 제거
tile_images = tile(remove_image_b)  # Patch로 나눔
concate_img = concate_images(tile_images)  # 다시 vis concate

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].imshow(remove_image_b)
axs[1].imshow(concate_img)

del remove_image_b, image_b, concate_img

### color normalization

일반적으로 Pathology image는 색의 분포가 다르다.
이를 맞춰주는 작업도 필요하다.(학습시 필수는 아니다)

In [ ]:
data_type = train["data_provider"].unique()
print(data_type)

kar_img_path = train[train["data_provider"] == data_type[0]].reset_index(drop=True).loc[0, "img_path"]
rad_img_path = train[train["data_provider"] == data_type[1]].reset_index(drop=True).loc[0, "img_path"]

kar_img = skimage.io.MultiImage(str(kar_img_path))
rad_img = skimage.io.MultiImage(str(rad_img_path))

remove_image_b = crop_white(np.array(kar_img)[0])
tile_kar_images = concate_images(tile(remove_image_b))

remove_image_b = crop_white(np.array(rad_img)[0])
tile_rad_images = concate_images(tile(remove_image_b))

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].imshow(tile_kar_images)
axs[1].imshow(tile_rad_images)

In [ ]:
normalizer = staintools.ReinhardColorNormalizer()
normalizer.fit(np.array(tile_kar_images))
reinhard_normalized = normalizer.transform(np.array(tile_rad_images))
fig, axs = plt.subplots(1, 2)
axs[0].imshow(tile_kar_images)
axs[1].imshow(reinhard_normalized)

In [ ]:
normalizer = staintools.ReinhardColorNormalizer()
normalizer.fit(np.array(tile_rad_images))
reinhard_normalized = normalizer.transform(np.array(tile_kar_images))
fig, axs = plt.subplots(1, 2)
axs[0].imshow(reinhard_normalized)
axs[1].imshow(tile_rad_images)